In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder
from scipy.optimize import minimize
from sklearn.metrics import classification_report
%matplotlib inline

# 数据载入与预处理

In [2]:
data = loadmat('ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
X, y = data['X'], data['y']
X.shape, y.shape

((5000, 400), (5000, 1))

## 多分类标签独热编码
> 可自己实现，可用现成库函数

In [4]:
# 自己实现
def expand_y(y):
    """expand 5000*1 into 5000*10
    where y=10 -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]: ndarray
    """
    result = []
    y = y.reshape(-1, 1)
    for k in range(1,11):
        result.append((y==k).astype(int))
    return np.squeeze(np.array(result).T)

In [5]:
# 使用库函数
encoder = OneHotEncoder(sparse=False, categories='auto')
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [6]:
y[0], y_onehot[0,:]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

# 前向传播

## sigmoid函数

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

## 初始化参数

In [8]:
def initialization(X, y, hidden_size=25):
    m = X.shape[1]
    label_size = y.shape[1]
    theta1 = (np.random.random((hidden_size, m+1)) - .5) * .25
    theta2 = (np.random.random((label_size, hidden_size+1)) - .5) * .25
    return theta1, theta2

## Forward propagation

> (400 + 1) -> (25 + 1) -> (10)

<img style="float: left;" src="../img/nn_model.png">

In [62]:
# theta1 (25, 401), theta2 (10,26)
def forward_propagation(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, np.ones(m), axis=1)  # 5000, 401
    z2 = a1 @ theta1.T  # 5000, 25
    a2 = np.insert(sigmoid(z2), 0, np.ones(m), axis=1)  # 5000, 26
    z3 = a2 @ theta2.T  # 5000，10
    h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [63]:
theta1, theta2 = initialization(X, y_onehot)

In [64]:
a1, z2, a2, z3, h = forward_propagation(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

# 代价函数 

<img style="float: left;" src="../img/nn_cost.png">

In [13]:
def cost(X, y, theta1, theta2):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    h = forward_propagation(X, theta1, theta2)[-1]
 
    pair_computation = -np.multiply(y, np.log(h))  - np.multiply((1 - y), np.log(1-h))
    
    return pair_computation.sum() / m

In [14]:
theta1, theta2 = initialization(X, y_onehot)
theta1.shape, theta2.shape

((25, 401), (10, 26))

In [15]:
%%time
cost(X, y_onehot, theta1, theta2)

Wall time: 34 ms


6.717900002930892

## 正则化代价函数
<img style="float: left;" src="../img/nn_regcost.png">

In [18]:
def regularized_cost(X, y, theta1, theta2, learning_rate = 1):
    m = X.shape[0]
    cost_term = cost(X, y, theta1, theta2)
    
    theta1_r = np.sum(np.power(theta1[:, 1:], 2))
    theta2_r = np.sum(np.power(theta2[:, 1:], 2))
    regularized_term = (learning_rate / (2 * m)) * (theta1_r + theta2_r)
    
    return cost_term + regularized_term

In [19]:
regularized_cost(X, y_onehot, theta1, theta2)

6.723301444733741

# 反向传播

In [20]:
# sigmoid激活函数的导数
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1-sigmoid(z)))

In [52]:
def back_propagation(X, y, theta1, theta2, learning_rate=1):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    a1, z2, a2, z3, h = forward_propagation(X, theta1, theta2)
    
    cost_ = regularized_cost(X, y, theta1, theta2, learning_rate)
    
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T @ d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return cost_, grad

In [53]:
theta1, theta2 = initialization(X, y_onehot)
theta1.shape, theta2.shape

((25, 401), (10, 26))

In [ ]:
cost_, grad = back_propagation(X, y_onehot, theta1, theta2)
cost_, grad.shape

> 上面的bp在minimize优化的时候不知道怎么传入参数= =，换一个

In [71]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate=1):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagation(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    J = regularized_cost(X, y, theta1, theta2)
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [68]:
# 初始化设置
input_size = 400
hidden_size = 25
num_labels = 10
learning_rate = 1

# 随机初始化完整网络参数大小的参数数组
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# 将参数数组解开为每个层的参数矩阵
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [72]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot)
J, grad.shape

(6.8693715225193825, (10285,))

In [73]:
def nn_training(params, input_size, hidden_size, num_labels, X, y):
    result = minimize(fun=backprop,
                     x0=params,
                     args=(input_size, hidden_size, num_labels, X, y),
                     method='TNC',
                     jac=True,
                     options={"maxiter": 250})
    return result

In [74]:
result = nn_training(params, input_size, hidden_size, num_labels, X, y_onehot)
result

     fun: 0.3368637955633817
     jac: array([-5.64434788e-04, -1.96711043e-06,  2.44557898e-06, ...,
       -8.72110602e-06, -1.95640853e-05,  5.09409700e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 20
  status: 3
 success: False
       x: array([ 0.08800653, -0.00983555,  0.01222789, ..., -2.85943584,
       -0.040507  , -1.07376352])

In [75]:
final_theta = result.x
final_theta.shape

(10285,)

In [76]:
theta1 = np.matrix(np.reshape(final_theta[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(final_theta[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
theta1.shape, theta2.shape

((25, 401), (10, 26))

In [81]:
y_pred = forward_propagation(X, theta1, theta2)[-1]
y_pred = np.array(np.argmax(y_pred, axis=1) + 1)
y_pred.shape

(5000, 1)

In [83]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           1       0.99      0.99      0.99       500
           2       1.00      0.99      0.99       500
           3       1.00      0.98      0.99       500
           4       0.99      0.99      0.99       500
           5       0.99      1.00      0.99       500
           6       1.00      1.00      1.00       500
           7       0.99      0.99      0.99       500
           8       0.99      1.00      1.00       500
           9       0.99      0.99      0.99       500
          10       1.00      1.00      1.00       500

    accuracy                           0.99      5000
   macro avg       0.99      0.99      0.99      5000
weighted avg       0.99      0.99      0.99      5000

